# Risk categories

In [2]:
!pip install --upgrade pip
!pip install pandas

In [1]:
import numpy as np
import pandas as pd
import pickle

import sys
sys.path.append('../../onc_functions')
from risk import get_risk_categories

print('python-' + sys.version)
import datetime
dte = datetime.datetime.now()
dte = dte.strftime("%Y%m%d")

python-3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]


In [2]:
with open('../complete_fairness_data.pickle', 'rb') as f:
        dataset = pickle.load(f)
dataset.head()

,usrds_id,died_in_90,inc_age,sex,dialtyp,race,hispanic,subset
0,2969676.0,0,72.0,2.0,1.0,2.0,2.0,8
1,2969869.0,1,73.0,2.0,1.0,2.0,2.0,9
2,2970021.0,0,55.0,2.0,1.0,1.0,1.0,8
3,2970067.0,0,72.0,1.0,1.0,1.0,2.0,9
4,2970686.0,0,49.0,2.0,1.0,2.0,2.0,8


In [3]:
# load results from the model
pred_df = pd.read_csv('./roc_auc/2021_xgb_nonimputed_y_proba.csv')
pred_df.drop(columns=['Unnamed: 0'],inplace=True)
pred_df.head()

,score,y,usrds_id
0,0.580939,1,31089.0
1,0.399797,0,34521.0
2,0.339976,0,46751.0
3,0.274023,0,50506.0
4,0.046709,0,54985.0


In [7]:
#merge the 2 dataframes from above 
data = pred_df.merge(dataset, on=['usrds_id'])
data.head()

,score,y,usrds_id,died_in_90,inc_age,sex,dialtyp,race,hispanic,subset
0,0.580939,1,31089.0,1,80.0,1.0,1.0,2.0,2.0,9
1,0.399797,0,34521.0,0,64.0,2.0,1.0,1.0,2.0,8
2,0.339976,0,46751.0,0,69.0,2.0,1.0,1.0,2.0,8
3,0.274023,0,50506.0,0,73.0,2.0,NaN,1.0,2.0,9
4,0.046709,0,54985.0,0,48.0,1.0,1.0,1.0,2.0,8


In [8]:
risk_cat = get_risk_categories(data,
                               y_proba_col_name='score',
                               y_true_col_name='died_in_90')
risk_cat

,Risk Category,Count,Count Died in 90,Count Survived,Percent Died in 90
0,0-0.09,148693,1702,146991,0.011446
1,0.1-0.19,75410,3256,72154,0.043177
2,0.2-0.29,44315,3722,40593,0.083990
3,0.3-0.39,29430,4064,25366,0.138090
4,0.4-0.49,20283,3893,16390,0.191934
5,0.5-0.59,13228,3399,9829,0.256955
6,0.6-0.69,7967,2679,5288,0.336262
7,0.7-0.79,4098,1827,2271,0.445827
8,0.8-0.89,1457,838,619,0.575154
9,0.9-0.99,417,351,66,0.841727


In [9]:
#save 
risk_cat.to_csv('./roc_auc/' + str(dte) + '_xgb_nonimputed_risk_cat.csv')